In [1]:
import pandas as pd
import pickle
import numpy as np
with open('phone_train.pickle', 'rb') as fh1:
    train = pickle.load(fh1)
with open('phone_test1.pickle', 'rb') as fh2:
    test = pickle.load(fh2)
x_train=np.array(train[train.columns[:-1]])
y_train=np.array(train["label"])
x_test=np.array(test[train.columns[:-1]])
y_test=np.array(test["label"])

In [2]:
# Q1.1: Create my mypgc class
import math
class mypgc:
    def __init__(self):
        self.position=["Phoneonleft","Phoneonfront","Phoneonback","Phoneonbottom","Phoneontop","Phoneonright"]
        self.attribute=["mu","cov","prec","detcov","n","prior"]
        self.info={}
        for p in self.position:
            self.info[p]={}
            for a in self.attribute:
                self.info[p][a]=None
    def fit(self, x, y):
        total=len(x)
        classes={}
#         index
        for i in self.position:
            classes[i]=np.where(y==i)[0]
#         x
        for i in self.position:
            classes[i]=x_train[classes[i],:]
#         mean, covariance, precision, det, n, prior    
        cov=0
        for i in self.position:
            self.info[i]["mu"]=np.mean(classes[i],axis=0)
            self.info[i]["cov"]=np.cov(classes[i],rowvar=False)
            self.info[i]["prec"]=np.linalg.pinv(self.info[i]["cov"])
            self.info[i]["detcov"]=np.log(np.linalg.det(self.info[i]["cov"]))
            self.info[i]["n"]=len(classes[i])
            self.info[i]["prior"]=len(classes[i])/total
            cov+=self.info[i]["cov"]*self.info[i]["prior"]
        inv=np.linalg.pinv(cov)
        self.checkinfo()
    def predict(self, xs):
        ans=[]
        for x in xs:
            ak=[]
            for i in self.position:
                det=self.info[i]["detcov"]
                cov=self.info[i]["cov"]
                mu=self.info[i]["mu"]
                prec=self.info[i]["prec"]
                prior=self.info[i]["prior"]
                p=-0.5*det+(-0.5*np.dot(np.dot((x-mu), prec),(x-mu).T))+np.log(prior)
                ak.append(p)
            index=np.argmax(ak, axis=0)
            ans.append(self.position[index])
        ans=np.array(ans)
        return ans
    def accuracy(self,ypred,y):
        return (np.sum(ypred==y)/len(y))
    def checkinfo(self):
        for p in self.position:
            print(p)
            for i in self.info[p]:
                if type(self.info[p][i])==type(np.array([])):
                    print("%s: \n"%i,str(self.info[p][i]))
                else:
                    print("%s: "%i,str(self.info[p][i]))
            print("-------------------------------------------------------------")

In [3]:
# Q1.2: train my model
pgc1 = mypgc()
pgc1.fit(x_train, y_train)

Phoneonleft
cov: 
 [[ 0.00192839 -0.00619274  0.00063551]
 [-0.00619274  0.03200844 -0.00317611]
 [ 0.00063551 -0.00317611  0.00171996]]
prec: 
 [[1369.95113577  263.0134811   -20.49603676]
 [ 263.0134811    88.74589895   66.6992188 ]
 [ -20.49603676   66.6992188   712.14869845]]
prior:  0.17667582302494958
n:  29522
detcov:  -17.232141935734493
mu: 
 [ 9.92639446  0.09275717 -0.01933473]
-------------------------------------------------------------
Phoneonfront
cov: 
 [[ 0.00173617 -0.00562249  0.00071319]
 [-0.00562249  0.03069977 -0.00386164]
 [ 0.00071319 -0.00386164  0.00185582]]
prec: 
 [[1415.5956848   258.4845593    -6.15206605]
 [ 258.4845593    91.32078358   90.68682325]
 [  -6.15206605   90.68682325  729.91124637]]
prior:  0.1740246683064328
n:  29079
detcov:  -17.331692277173115
mu: 
 [ 0.11270401  0.14265129 -9.73579633]
-------------------------------------------------------------
Phoneonback
cov: 
 [[ 0.00299006 -0.00195813  0.00284766]
 [-0.00195813  0.00229887 -0.00242

In [4]:
# Q1.3: List the first 20 predictions and well as the correct labels
#       Compute the accuracy for your model
ypred=pgc1.predict(x_test)
print("accuracy: ",pgc1.accuracy(ypred,y_test))
print("first 20: \n",ypred[0:20])
print("correct 20: \n",y_test[0:20])

accuracy:  1.0
first 20: 
 ['Phoneonfront' 'Phoneonbottom' 'Phoneonbottom' 'Phoneonbottom'
 'Phoneonfront' 'Phoneonright' 'Phoneonfront' 'Phoneontop' 'Phoneonfront'
 'Phoneonfront' 'Phoneonfront' 'Phoneonback' 'Phoneonleft' 'Phoneonback'
 'Phoneonbottom' 'Phoneonback' 'Phoneonright' 'Phoneonright' 'Phoneontop'
 'Phoneonleft']
correct 20: 
 ['Phoneonfront' 'Phoneonbottom' 'Phoneonbottom' 'Phoneonbottom'
 'Phoneonfront' 'Phoneonright' 'Phoneonfront' 'Phoneontop' 'Phoneonfront'
 'Phoneonfront' 'Phoneonfront' 'Phoneonback' 'Phoneonleft' 'Phoneonback'
 'Phoneonbottom' 'Phoneonback' 'Phoneonright' 'Phoneonright' 'Phoneontop'
 'Phoneonleft']


In [1]:
# Q2.1
# 先看一下資料的大小和長相
import pandas as pd
import numpy as np
adult=pd.read_csv('adult.csv',header = None)
adult_test=pd.read_csv('adult_test.csv',header = None)
print("train:", adult.shape)
print("test:", adult_test.shape)
adult.head()

train: (32561, 15)
test: (16281, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
# 看一下數值資料
adult.columns =adult_test.columns= ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
'marital_status', 'occupation', 'relationship',
'race', 'sex', 'capital_gain', 'capital_loss',
'hours_per_week', 'native_country', 'income']
adult.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [3]:
# 是否有缺值
adult.info()
# adult_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
# 沒有缺值是否有以其他符號代表缺值
notnum=[ 'workclass', 'education',
'marital_status', 'occupation', 'relationship',
'race', 'sex' , 'native_country', 'income']
for x in notnum:
    print(adult[x].unique())

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
[' Male' ' Female']
[' United-States' ' Cuba' ' Jamaica' ' India' ' ?' ' Mexico' ' South'
 ' Puerto-Rico' ' Honduras' ' England' ' Canada' ' Germany' 

In [5]:
# 處理'？'
for x in notnum :
    print(x , ":" , sum(adult[x]==" ?"))
    print(x , ":" , sum(adult_test[x]==" ?"))
# 把問號換成缺值
adult=adult.replace(' ?',np.nan)
adult=adult.dropna()
adult.shape

workclass : 1836
workclass : 963
education : 0
education : 0
marital_status : 0
marital_status : 0
occupation : 1843
occupation : 966
relationship : 0
relationship : 0
race : 0
race : 0
sex : 0
sex : 0
native_country : 583
native_country : 274
income : 0
income : 0


(30162, 15)

In [6]:
# 處理label
def encode(df):
    x=pd.DataFrame.copy(df)
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder
    x['income']=LabelEncoder().fit_transform(x['income'])
    x=pd.get_dummies(x)
    return x
adult_clean=encode(adult)
adult_test_clean=encode(adult_test)
adult_clean.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 把頻率小於10的拿掉
print(len(adult_clean.columns))
# for i in adult_clean.columns:
#     print(i, adult_clean[i].sum())
adult_clean=adult_clean.drop(columns=[i for i in adult_clean.columns if adult_clean[i].sum()<=10])
print(len(adult_test_clean.columns))
adult_test_clean=adult_test_clean[adult_clean.columns]
print(len(adult_test_clean.columns))
print(len(adult_clean.columns))

105
108
103
103


In [8]:
x_train=np.array(adult_clean.drop(columns=['income']))
y_train=np.array(adult_clean['income'])
x_test=np.array(adult_test_clean.drop(columns=['income']))
y_test=np.array(adult_test_clean['income'])

In [9]:
def sigmoid(z):
        z_p = 1/(1.0 + np.exp(-z))
        return z_p

In [14]:
# Q2.2 Derive the gradient and hession matrix for the new E(w)
# 用講義裡gradient和hessian的公式加上 L2項的微分和二次微分
dim=len(x_train[0])
lambda_vec=np.identity(dim)
b=np.sum(lambda_vec)/dim
w=np.dot(np.dot(np.linalg.pinv(np.dot(x_train.T,x_train)+b*np.identity(dim)),x_train.T),y_train)
y = sigmoid(np.dot(x_train, w))
R=np.identity(len(y))
np.fill_diagonal(R,y*(1-y))
H=np.dot(np.dot(x_train.T, R),x_train)+lambda_vec
g=np.dot(x_train.T,y-y_train)+np.dot(lambda_vec,w)
# print(np.dot(lambda_vec,w).shape)
# print(lambda_vec.shape)
print(H.shape)
print(g.shape)
print("gradient: \n",g)
print("hession: \n",H)

(102, 102)
(102,)
gradient: 
 [ 3.29992296e+05  1.79383014e+09  8.66823581e+04 -6.47332098e+06
  2.27657005e+05  3.57651244e+05  1.95430543e+02  5.73261389e+02
  7.45787118e+03  8.02104774e+01  7.09528076e+02  3.79256708e+02
  6.92722379e+00  3.65480559e+02  4.79503590e+02  1.66615719e+02
  7.09809363e+01  1.34948665e+02  2.52116011e+02  2.08630210e+02
  3.10882218e+02  3.94133741e+02  9.13149421e+02 -2.63354267e+01
  3.70177898e+03  1.16477393e+02  2.25207941e+01 -3.98540571e+01
  2.33145686e+03  1.76666996e+03  3.02291978e+00  2.18918705e+03
  1.61589865e+02  4.50900932e+03  4.19733526e+02  3.53272976e+02
  1.48511060e+03  1.33131688e+03  5.27305815e+02  4.07878899e+02
  6.12472965e+02  7.98610489e+02  1.50640153e+03  7.07779722e+01
  6.44632490e+02  1.63685734e+02  1.05901998e+03  2.46277903e+02
  5.45889601e+02  1.93300954e+03  3.24327942e+03  4.18068281e+02
  2.18469246e+03  1.44561513e+03  1.77820774e+02  1.17309633e+02
  2.59758570e+02  1.13226782e+03  9.95008401e+01  7.79364874

In [30]:
# Q2.3 Create your own mylogistic_l2 class
class mylogistic_l2:
    def __init__(self, reg_vec, max_iter = 1000, tol = 1e-5, add_intercept = True):
        self.reg_vec=reg_vec
        self.max_iter=max_iter
        self.tol=tol
        self.add_intercept=add_intercept
        self.w=None
    def error(self, yreal, yprob):
        e=-1*np.sum(np.dot(yreal, np.log(yprob+1e-10))+np.dot((1-yreal.T), np.log(1-yprob+1e-10)))
        l2=0.5*np.dot(np.dot(self.w.T, self.reg_vec), self.w)
        return e+l2
    def fit(self, x_train, y_train):
        if self.add_intercept:
            x_train=np.concatenate((x_train, np.ones((x_train.shape[0],1))), axis=1)
        else:
            self.reg_vec=np.delete(self.reg_vec, np.s_[-1], axis=0)
            self.reg_vec=np.delete(self.reg_vec, np.s_[-1], axis=1)
        dim=len(x_train[0])
        b=np.sum(self.reg_vec)/dim
        self.w=np.dot(np.dot(np.linalg.pinv(np.dot(x_train.T,x_train)+b*np.identity(dim)),x_train.T),y_train)
        y = sigmoid(np.dot(x_train, self.w))
        e=self.error(y_train, y)
        print(e)
        for i in range(self.max_iter):
            R=np.identity(len(y))
            np.fill_diagonal(R,y*(1-y))
            H=np.dot(np.dot(x_train.T, R),x_train)+lambda_vec
            g=np.dot(x_train.T,y-y_train)+np.dot(lambda_vec,self.w)
            w_new=self.w-np.dot(np.linalg.pinv(H),g)
            y = sigmoid(np.dot(x_train, w_new))
            e_new=self.error(y_train, y)
            print(e_new)
            if e-e_new<self.tol:
                break
            e=e_new
            self.w=w_new
#         print(self.w.shape)
#         print("w: \n",self.w)
    def predict(self, x_test):
        if self.add_intercept:
            x_test=np.concatenate((x_test, np.ones((x_test.shape[0],1))), axis=1)
        return np.where(sigmoid(np.dot(x_test, self.w)) <= 0.5, 0, 1)
    def accuracy(self, ypred, y):
        return (np.sum(ypred==y)/len(y))

In [20]:
# case1
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.90948300757
11772.152130565039
10288.924339338619
9847.945218608545
9782.60079217587
9783.16562521719
w: 
 [ 2.45276265e-02  7.22623816e-07  1.82932216e-01  3.02984037e-04
  6.33868046e-04  2.86400999e-02  1.97247701e-01 -4.83788292e-01
 -2.98785498e-01 -1.15976711e-01 -7.77472593e-01 -6.04570339e-01
 -1.17117985e+00 -4.56471758e-01 -5.28430884e-01 -3.76154118e-01
 -9.27409978e-02 -2.53355851e-01 -6.30258675e-01 -5.03252463e-01
 -3.05490356e-01 -1.29901768e-01  1.25969611e-01  5.86389169e-01
 -2.54942139e-01  3.00245441e-01 -1.30745096e+00  6.74624567e-01
 -1.03304398e-01 -9.63080329e-01  1.10620435e+00  7.37575882e-01
 -9.00793970e-01 -1.43120238e+00 -1.03033523e+00 -7.72893897e-01
 -9.92047140e-02 -4.08064951e-02  6.95406631e-01 -1.07137021e+00
 -7.74722290e-01 -3.66143240e-01 -8.76877761e-01 -1.47661415e+00
  4.12377562e-01  4.80903597e-01  1.91532623e-01  5.48337499e-01
 -1.94647642e-01 -5.20047151e-01 -4.75559665e-01 -1.09351700e+00
 -1.33857883e+00 -5.93558900e-01  7.667359

In [21]:
# case2
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.89239880735
11770.745593350817
10284.321955803034
9839.18894135134
9770.019529630285
9768.768097759195
9769.872239243798
w: 
 [ 2.54168388e-02  7.50575253e-07  2.95032891e-01  3.16447271e-04
  6.39438248e-04  2.94732166e-02  7.05194032e-01  1.75442865e-02
  2.08776908e-01  3.82505442e-01 -2.79964123e-01 -1.04857877e-01
 -9.29838990e-01  8.95398892e-02 -1.06384704e-01 -5.71560278e-02
  7.06734123e-01  5.35311639e-01  1.15048424e-01  1.35975084e-01
 -4.01614183e-01 -1.13522859e-01 -7.41687063e-02  6.87050909e-02
 -2.01587670e-02 -1.10173711e-02 -1.15682617e+00  2.67370384e-01
  2.15238292e-02 -5.22178089e-01  1.60264646e+00  1.35497972e+00
 -4.87621381e-01 -1.00956735e+00 -6.01248235e-01 -3.37651446e-01
  1.62624002e-01  2.26755089e-01  9.63025054e-01 -8.19194369e-01
 -5.21940414e-01 -1.00607862e-01 -6.49400879e-01 -1.53538645e+00
  6.76704706e-01  7.49304793e-01  4.53833531e-01  8.16870884e-01
  7.15601662e-02 -3.82636769e-02  1.85911690e-01 -5.83727942e-01
 -9.16898134e-01  6.208

In [22]:
# case3
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)*0.5
for i in range(6):
    lambda_vec[i][i]=1
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy: ",logic1.accuracy(ypred,y_test))

21190.655269525745
11770.26884256605
10283.062347187773
9836.233456645698
9762.439376531358
9759.514928122415
9760.978055786567
w: 
 [ 2.54578068e-02  7.51895502e-07  3.18963632e-01  3.16721802e-04
  6.39884166e-04  2.94931417e-02  7.63391195e-01  7.29144363e-02
  2.64852429e-01  4.39789105e-01 -2.24068364e-01 -4.97999576e-02
 -1.29029985e+00  2.18877359e-01 -1.47070170e-03  2.26700065e-02
  9.79062628e-01  7.55586814e-01  2.92475145e-01  2.93120221e-01
 -4.19265805e-01 -1.04326459e-01 -1.12957527e-01 -3.78844553e-02
  3.75695779e-02 -7.28515543e-02 -2.11476589e+00  1.86053364e-01
  5.48862692e-02 -5.70327020e-01  1.80986043e+00  1.37847046e+00
 -5.44777223e-01 -1.05624528e+00 -6.53737611e-01 -3.86464773e-01
  2.46084350e-01  3.09929806e-01  1.04777876e+00 -7.42504459e-01
 -4.43234436e-01 -1.69667703e-02 -5.71031584e-01 -1.91034970e+00
  7.60674646e-01  8.36992630e-01  5.38109580e-01  9.04519295e-01
  1.54927523e-01 -8.23024109e-02  2.13697571e-01 -5.96145776e-01
 -9.05449635e-01  9.24

In [26]:
# Q2.4
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
x_subtrain, x_tuning, y_subtrain, y_tuning = train_test_split(x_train, y_train, test_size=0.1, random_state=87)
print(len(x_subtrain))
print(len(y_subtrain))
print(len(x_tuning))
print(len(x_tuning))

27145
27145
3017
3017


In [34]:
# Choose a set of grids among a reasonable range. For example, 10 grids in [0.01, 100]
pick=[0.01*(10**(4/9))**i for i in range(10)]
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy[%f]:"%a,logic1.accuracy(ypred,y_tuning))
a1=pick[np.argmax(temp, axis=0)]
print(a1)

19057.501243572206
10580.215672294786
9237.748496448572
8827.996794544937
8757.704597293381
8749.98818436054
8750.223992999672
accuracy[0.010000]: 0.8485250248591316
19057.512582739335
10580.261427755704
9237.906068262555
8828.335308326361
8758.417220492085
8751.988610679606
8751.26066502644
8751.743409730569
accuracy[0.027826]: 0.8485250248591316
19057.544080227264
10580.388860976476
9238.343457013782
8829.2969443626
8760.300687821933
8753.560243187254
8756.021597925695
accuracy[0.077426]: 0.8485250248591316
19057.631076611073
10580.71785019958
9239.391550872053
8831.77528539194
8764.886292495383
8760.942364181628
8762.621692938603
accuracy[0.215443]: 0.8485250248591316
19057.870971564684
10581.565237580215
9242.026704591593
8834.924379378814
8770.208270912759
8768.271465941129
8769.575575600791
accuracy[0.599484]: 0.8488564799469672
19058.477731197945
10582.972539660474
9246.70899357559
8845.088555666014
8784.763516860516
8784.771999514183
accuracy[1.668101]: 0.8485250248591316
19060

In [35]:
# Conduct grid search with the constraint that  a1=a2
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a
    for i in range(6):
        lambda_vec[i][i]=a1
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy[%f]:"%a,logic1.accuracy(ypred,y_tuning))
a2=pick[np.argmax(temp, axis=0)]
print(a2)

19057.515766234726
10580.217155558828
9237.752293231468
8828.004597615954
8757.724275114715
8750.00979118866
8750.281419686202
accuracy[0.010000]: 0.8485250248591316
19057.52665286035
10580.262859964037
9237.909751034276
8828.342861191137
8758.43578479344
8752.038255333182
8751.271193610228
8751.785525578845
accuracy[0.027826]: 0.8485250248591316
19057.55689573259
10580.390150051919
9238.346820238672
8829.30380357321
8760.316406321646
8753.57755745307
8756.039178375911
accuracy[0.077426]: 0.8485250248591316
19057.64036530703
10580.708559927964
9239.39979649589
8831.78419231772
8764.895419413855
8760.956741583903
8762.63275940501
accuracy[0.215443]: 0.8485250248591316
19057.870971564684
10581.565237580215
9242.026704591593
8834.924379378814
8770.208270912759
8768.271465941129
8769.575575600791
accuracy[0.599484]: 0.8488564799469672
19058.452831862473
10582.970129765597
9246.700498021139
8845.060009297096
8784.723760189501
8784.726966938008
accuracy[1.668101]: 0.8485250248591316
19060.08

In [36]:
# Fix a1, and search a2
temp=[]
for a in pick:
    lambda_vec=np.identity(dim+1)*a2
    for i in range(6):
        lambda_vec[i][i]=a
    lambda_vec[dim][dim]=0
    logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic1.fit(x_subtrain, y_subtrain)
    ypred = logic1.predict(x_tuning)
    temp.append(logic1.accuracy(ypred,y_tuning))
    print("accuracy[%f]:"%a,logic1.accuracy(ypred,y_tuning))
a1=pick[np.argmax(temp, axis=0)]
print(a1)

19057.85686455127
10581.564081755881
9242.031759545922
8834.903249368543
8770.173531029412
8768.246101279601
8769.545887983842
accuracy[0.010000]: 0.8488564799469672
19057.857291081007
10581.564116463263
9242.031912722332
8834.903700379853
8770.174277094522
8768.247059408903
8769.54680304319
accuracy[0.027826]: 0.8488564799469672
19057.85847814735
10581.5642141703
9242.032341680286
8834.904954943364
8770.176352684315
8768.24972411652
8769.549348234981
accuracy[0.077426]: 0.8488564799469672
19057.861782461376
10581.56449107775
9242.033547541874
8834.908442723286
8770.182125114245
8768.257128528663
8769.556422486956
accuracy[0.215443]: 0.8488564799469672
19057.870971564684
10581.565237580215
9242.026704591593
8834.924379378814
8770.208270912759
8768.271465941129
8769.575575600791
accuracy[0.599484]: 0.8488564799469672
19057.89651160158
10581.567377861245
9242.046108376657
8834.944855585405
8770.24260592991
8768.334157882398
8769.630182509249
accuracy[1.668101]: 0.8488564799469672
19057.8

In [37]:
print("best a1 and a2: ",a1,a2)
dim=len(x_train[0])
lambda_vec=np.identity(dim+1)*a2
for i in range(6):
    lambda_vec[i][i]=a1
lambda_vec[dim][dim]=0
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("accuracy[%f]:"%a,logic1.accuracy(ypred,y_test))

best a1 and a2:  0.01 0.5994842503189409
21190.690725489174
11770.545166596938
10283.899735554693
9837.704269446105
9764.079326577887
9761.637131525962
9762.884421887391
accuracy[100.000000]: 0.8526503286038941


In [40]:
# Q2.5 
from sklearn.linear_model import LogisticRegression

pick=[0.01*(10**(4/9))**i for i in range(10)]
temp=[]
for a in pick:
    classifier=(LogisticRegression(C=1/a,tol=1e-5,max_iter=1000,n_jobs=2,solver='newton-cg'))
    classifier.fit(x_train,y_train)
    ypred=classifier.predict(x_test)
    acc=np.sum(ypred==y_test)/len(y_test)
    print("accuracy[%f]:"%a,acc)
    temp.append(acc)
a=pick[np.argmax(temp, axis=0)]
print("best a:",a)

accuracy[0.010000]: 0.8525889073152755
accuracy[0.027826]: 0.8525274860266568
accuracy[0.077426]: 0.8525889073152755
accuracy[0.215443]: 0.8524046434494196
accuracy[0.599484]: 0.8526503286038941
accuracy[1.668101]: 0.8528960137583687
accuracy[4.641589]: 0.8536330692217923
accuracy[12.915497]: 0.8532031202014618
accuracy[35.938137]: 0.8520361157177078
accuracy[100.000000]: 0.8508691112339537
best a: 4.641588833612778


In [ ]:
'''
比較：
用套件挑出來的參數和我寫的logistic regression挑出來的不太一樣
不過套件調參數的時候不能分binary和constant給不一樣的lambda
所以其實不太能一起比較
不過結果有差不多的accuracy

'''